In [10]:
from models import LatentClassifier, VAE_CELL_CNN

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# mnist dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

from dataset_tools import OwnDataset, make_train_test_val_split
#from plotting import show_img, loss_plots, accuracy_plots, plot_random_images, plot_image_comparison, plot_latent
from collections import defaultdict


from statsmodels.stats.contingency_tables import mcnemar
from models import LatentClassifier, VAE_CELL_CNN, CELL_CNN_CLASSIFIER, CELL_CNN_CLASSIFIER_2


cuda = torch.cuda.is_available()
print(cuda)

False


# Loading Data 

In [49]:
val = True
tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x.view(-1)) # notice that we dont flatten when we are going to use CNN
])
dataset = OwnDataset(transform=tf, path = r"C:\Users\vlbr1\Downloads\labelled_data-20240618T120814Z-001\labelled_data")
# dataset = OwnDataset(transform=tf)
train_subset, test_subset, val_subset = make_train_test_val_split(dataset)


train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=True)

print(len(dataset), len(train_loader), len(test_loader), len(val_loader))



2684 27 9 7


# Loading Models 

In [45]:

from models import LatentClassifier_2, VAE_CELL_CNN, CELL_CNN_CLASSIFIER, CELL_CNN_CLASSIFIER_2, VAE_CELL_CNN
from loss_functions import loss_function
# import mse loss


# now we load the saved models into the model class
MODEL_DIR = "trained_models/"


latent_dim_256 = LatentClassifier_2(hidden_dim=512, latent_dim=256, num_classes=13)
latent_dim_256.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_256.__class__.__name__}_latent{latent_dim_256.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))

latent_dim_2 = LatentClassifier_2(hidden_dim=4, latent_dim=2, num_classes=13)
latent_dim_2.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_2.__class__.__name__}_latent{latent_dim_2.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))

Cell_CNN_Classifier = CELL_CNN_CLASSIFIER(input_dim=(3,68,68),hidden_dim=128, num_classes=13)
Cell_CNN_Classifier.load_state_dict(torch.load(MODEL_DIR + f"{Cell_CNN_Classifier.__class__.__name__}_best_model.pth",map_location=torch.device('cpu')))

Vae_Cell_CNN_256 = VAE_CELL_CNN(input_dim=(3,68,68), hidden_dim=512, latent_dim=256)
Vae_Cell_CNN_256.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_256.__class__.__name__}_latent{Vae_Cell_CNN_256.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))


Vae_Cell_CNN_2 = VAE_CELL_CNN(input_dim=(3,68,68), hidden_dim=4, latent_dim=2)
Vae_Cell_CNN_2.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_2.__class__.__name__}_latent{Vae_Cell_CNN_2.latent_dim}_mean_best_model.pth",map_location=torch.device('cpu')))


# models = [vae_cnn_own]
models = [latent_dim_256, latent_dim_2, Cell_CNN_Classifier]
# models = [ae_cnn_own]



Comparing the classification performance across trained models

In [57]:
batch_accuracy_256_latent = []
batch_accuracy_2_latent = []
batch_accuracy_CNN = []
for image, moa in train_loader:
    print(Cell_CNN_Classifier(image))
    print(moa)
    print("_________")

    
    # for 256 dim latent space
    latent_output = Vae_Cell_CNN_256(image)
    x_hat, mu, sigma = latent_output["x_hat"], latent_output["mu"], latent_output["sigma"]

    pred_256 = latent_dim_256(mu)
    batch_accuracy_256_latent.append(pred_256.argmax(1)== moa)


    # for 2 dim latent space
    latent_output = Vae_Cell_CNN_2(image)
    x_hat, mu, sigma = latent_output["x_hat"], latent_output["mu"], latent_output["sigma"]

    pred_2 = latent_dim_2(mu)
    batch_accuracy_2_latent.append(pred_2.argmax(1)== moa)


    #Cell_CNN_Classifier(image) == moa
    batch_accuracy_CNN.append((Cell_CNN_Classifier(image).argmax(1) == moa))



    print(batch_accuracy_CNN)
    print(batch_accuracy_256_latent)
    print(batch_accuracy_2_latent)
    break
    
    
 # Converting to arrays
batch_accuracy_CNN = np.concatenate(batch_accuracy_CNN)
batch_accuracy_256_latent = np.concatenate(batch_accuracy_256_latent)
batch_accuracy_2_latent = np.concatenate(batch_accuracy_2_latent)

tensor([[-1.3064e+01, -4.2895e+00, -3.1707e-01,  4.2717e+00, -2.0705e+00,
         -9.4312e-01,  7.1979e-01,  5.3047e+00, -5.3909e+00, -5.4827e+00,
         -2.0141e+00, -3.4554e+00, -3.2887e+00],
        [-6.8118e+00, -7.3749e+00, -7.0125e+00,  9.0711e-01, -5.3419e+00,
          3.2984e+00, -6.3171e-01, -2.8010e+00, -1.4894e+01, -3.8399e+00,
          1.0398e+00,  2.1047e+00, -9.2228e+00],
        [-1.3795e+01, -2.3826e+00, -2.8206e+00,  6.3322e+00, -1.0315e+00,
         -3.3639e+00, -6.8899e+00, -1.4374e+00,  6.6311e+00, -5.6284e+00,
         -4.7551e+00, -7.0501e+00, -1.0151e+01],
        [-1.7463e+00, -1.2792e+00, -6.1706e+00, -6.0293e-01, -3.3725e+00,
         -3.4480e+00, -2.7904e-01, -9.0547e+00, -9.9473e+00, -2.1528e+00,
          8.1426e+00, -8.4032e-01, -6.0376e+00],
        [-1.1608e+01,  3.9443e-03, -4.8261e+00,  3.2513e+00,  1.4192e+00,
          4.9769e-02, -2.9722e+00, -2.0978e-01, -8.5203e+00, -4.8076e+00,
         -4.5128e+00, -1.1885e+00, -4.8123e+00],
        [-1.244

# Confusion matrix + Macnemar Test 

# Latent 256 vs CNN

In [58]:


n00 = n01 = n10 = n11 = 0

for i in range(len(batch_accuracy_CNN)):
    if batch_accuracy_CNN[i] and batch_accuracy_256_latent[i]:
        n11 += 1
    elif batch_accuracy_CNN[i] and not batch_accuracy_256_latent[i]:
        n10 += 1
    elif not batch_accuracy_CNN[i] and batch_accuracy_256_latent[i]:
        n01 += 1
    else:
        n00 += 1

confusion_matrix = [[n00, n01], [n10, n11]]
print(confusion_matrix)

result = mcnemar(confusion_matrix, exact=True)
print(f'statistic={result.statistic}, p-value={result.pvalue}')

alpha = 0.05
if result.pvalue < alpha:
    print('Significant difference in performance (reject H0)')
else:
    print('No significant difference in performance (fail to reject H0)')


[[21, 1], [24, 18]]
statistic=1.0, p-value=1.5497207641601562e-06
Significant difference in performance (reject H0)


# Latent 256 vs Latent 2 

In [59]:
n00 = n01 = n10 = n11 = 0
# latent 256 vs latent 2 

for i in range(len(batch_accuracy_256_latent)):
    if batch_accuracy_256_latent[i] and batch_accuracy_2_latent[i]:
        n11 += 1
    elif batch_accuracy_256_latent[i] and not batch_accuracy_2_latent[i]:
        n10 += 1
    elif not batch_accuracy_256_latent[i] and batch_accuracy_2_latent[i]:
        n01 += 1
    else:
        n00 += 1

confusion_matrix = [[n00, n01], [n10, n11]]

result = mcnemar(confusion_matrix, exact=True)
print(f'statistic={result.statistic}, p-value={result.pvalue}')

alpha = 0.05
if result.pvalue < alpha:
    print('Significant difference in performance (reject H0)')
else:
    print('No significant difference in performance (fail to reject H0)')


statistic=1.0, p-value=0.000518798828125
Significant difference in performance (reject H0)


# Latent 2 vc CNN 

In [60]:
n00 = n01 = n10 = n11 = 0
# latent 2 vs CNN

for i in range(len(batch_accuracy_2_latent)):
    if batch_accuracy_2_latent[i] and batch_accuracy_CNN[i]:
        n11 += 1
    elif batch_accuracy_2_latent[i] and not batch_accuracy_CNN[i]:
        n10 += 1
    elif not batch_accuracy_2_latent[i] and batch_accuracy_CNN[i]:
        n01 += 1
    else:
        n00 += 1


confusion_matrix = [[n00, n01], [n10, n11]]

result = mcnemar(confusion_matrix, exact=True)
print(f'statistic={result.statistic}, p-value={result.pvalue}')

alpha = 0.05
if result.pvalue < alpha:
    print('Significant difference in performance (reject H0)')
else:
    print('No significant difference in performance (fail to reject H0)')


statistic=0.0, p-value=1.4551915228366852e-11
Significant difference in performance (reject H0)
